In [1]:
# import
import importlib
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

/Users/johnbergmann/.pyenv/versions/3.10.11/envs/RB_NLP/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/johnbergmann/.pyenv/versions/3.10.11/envs/RB_NLP/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/johnbergmann/.pyenv/v

In [2]:
# get data
topic = "Twitter"
pipeline_executor = PipelineExecutor()
articles = pipeline_executor.execute(query=topic, max_articles=None, overwrite=False)

Getting news article info:   1%|          | 5/556 [00:00<00:51, 10.76it/s]encoding error : input conversion failed due to input error, bytes 0x21 0x00 0x00 0x00
encoding error : input conversion failed due to input error, bytes 0x44 0x00 0x00 0x00
I/O error : encoder error
Getting news article info:   1%|▏         | 7/556 [00:00<00:48, 11.21it/s]encoding error : input conversion failed due to input error, bytes 0x21 0x00 0x00 0x00
encoding error : input conversion failed due to input error, bytes 0x44 0x00 0x00 0x00
I/O error : encoder error
Getting news article info:   3%|▎         | 15/556 [00:01<00:47, 11.45it/s]encoding error : input conversion failed due to input error, bytes 0x21 0x00 0x00 0x00
encoding error : input conversion failed due to input error, bytes 0x64 0x00 0x00 0x00
I/O error : encoder error
Getting news article info:   5%|▍         | 26/556 [00:02<00:41, 12.76it/s]encoding error : input conversion failed due to input error, bytes 0x21 0x00 0x00 0x00
encoding error 

In [3]:
# Paragraphs - Word - silhouette
wizard = ww.WordWizard(articles)

In [4]:
wizard.create_sentence_embeddings()

Batches:   0%|          | 0/76 [00:00<?, ?it/s]

In [5]:
wizard.cluster_embeddings()

In [6]:
wizard.entitiy_recognition()

Extracting organizations from paragraph:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
wizard.topic_modelling()

In [9]:
wizard.find_sentiment()

Calculating sentiment using paragraph:   0%|          | 0/2197 [00:00<?, ?it/s]

In [10]:
wizard.reduce_demensionality()

In [11]:
# topics comes as a list of tuples (topic, score). I want to keep only the first 5 tuples and round the score to 3 decimals
wizard.df["topics"] = wizard.df["topics"].apply(lambda x: [tuple((topic, round(score, 3))) for topic, score in x[:5]])

In [13]:
wizard_copy = wizard.df.copy()

In [14]:
wizard.df['x'] = wizard.df.iloc[:,-1].apply(lambda x: x[0])
wizard.df['y'] = wizard.df.iloc[:,-2].apply(lambda x: x[1])

In [15]:
# count cluster size
wizard.df['cluster_size'] = wizard.df.groupby('paragraph_sentence_embeddings_clusters')['paragraph_sentence_embeddings_clusters'].transform('count')

In [18]:
# aggregate cluster sentiment
wizard.df['cluster_sentiment'] = wizard.df.groupby('paragraph_sentence_embeddings_clusters')['paragraph_sentiment'].transform('mean')

In [19]:
# keep only medoids
wizard.df = wizard.df[wizard.df['paragraph_sentence_embeddings_clusters_medoids'] == True]

In [20]:
# keep only cluster_size, x, y, cluster_sentiment, paragraph_clusters_sentence_embeddings_NER, topics
ner_col = [col for col in wizard.df.columns if col.endswith('NER')][0]
wizard.df = wizard.df[['cluster_size', 'x', 'y', 'cluster_sentiment', ner_col, 'topics']]

In [21]:
# rename columns
wizard.df.columns = ['size', 'x', 'y', 'sentiment', 'entities', 'topics']

In [23]:
wizard.df

,size,x,y,sentiment,entities,topics
131,565,0.831824,16.119192,0.472566,"['Twitter', 'Fox News', 'Raw Videos', 'NPR', '...","[(oconnor, 0.01), (said, 0.009), (year, 0.007)..."
381,158,-1.159285,2.252434,1.094937,['“Obviously I didn’t get the big score I’d ha...,"[(ball, 0.026), (england, 0.025), (smith, 0.01..."
395,158,-1.102350,2.041899,1.094937,['“Obviously I didn’t get the big score I’d ha...,"[(ball, 0.026), (england, 0.025), (smith, 0.01..."
449,608,2.013791,9.731251,1.067434,"['Twitter', 'In a virtual memorial service, th...","[(honor, 0.008), (technology, 0.007), (ai, 0.0..."
1031,608,0.271912,6.411808,1.067434,"['Twitter', 'In a virtual memorial service, th...","[(honor, 0.008), (technology, 0.007), (ai, 0.0..."
1525,941,-3.882543,13.117978,0.646121,"['Twitter', 'Google', 'NPR', 'EU', 'Big Tech']","[(musk, 0.015), (twitter, 0.014), (account, 0...."
1553,140,-2.096040,4.112012,1.257143,"['NBA', 'Rangers', 'Cardinals', 'NCAA', ""How E...","[(season, 0.026), (team, 0.021), (game, 0.017)..."
1736,565,-2.806425,9.852136,0.472566,"['Twitter', 'Fox News', 'Raw Videos', 'NPR', '...","[(oconnor, 0.01), (said, 0.009), (year, 0.007)..."
2204,140,-2.142520,4.102574,1.257143,"['NBA', 'Rangers', 'Cardinals', 'NCAA', ""How E...","[(season, 0.026), (team, 0.021), (game, 0.017)..."
2337,941,-2.887728,12.163920,0.646121,"['Twitter', 'Google', 'NPR', 'EU', 'Big Tech']","[(musk, 0.015), (twitter, 0.014), (account, 0...."


In [24]:
# safe to csv
wizard.df.to_csv('twitter.csv', index=False)